<a href="https://colab.research.google.com/github/VasylDvorakDS/Picture_recognizing_with_CNN_-TPU_Strategy/blob/master/Picture_recognizing_with_CNN__TPU_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:


import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, \
  GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model



In [3]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [4]:
strategy = tf.distribute.TPUStrategy(resolver)

In [5]:
def create_model():
  i = Input(shape=(32, 32, 3))

  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(10)(x)

  model = Model(i, x)
  return model

In [6]:


# Load in the data
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape:", x_train.shape)
print("y_train.shape", y_train.shape)



170498071/170498071 [==============================] - 2s 0us/step
x_train.shape: (50000, 32, 32, 3)
y_train.shape (50000,)


In [7]:


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))



In [8]:
with strategy.scope():
  model = create_model()
  model.compile(
      optimizer='adam',
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['sparse_categorical_accuracy'])

batch_size = 256

# reshuffle_each_iteration=None is default but is later set to True if None
# thus "True" is the actual default
train_dataset = train_dataset.shuffle(1000).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset)

Epoch 1/5
196/196 [==============================] - 22s 68ms/step - loss: 1.7020 - sparse_categorical_accuracy: 0.4622 - val_loss: 3.2460 - val_sparse_categorical_accuracy: 0.1351
Epoch 2/5
196/196 [==============================] - 7s 34ms/step - loss: 0.9944 - sparse_categorical_accuracy: 0.6461 - val_loss: 2.6267 - val_sparse_categorical_accuracy: 0.2917
Epoch 3/5
196/196 [==============================] - 7s 35ms/step - loss: 0.7864 - sparse_categorical_accuracy: 0.7227 - val_loss: 0.8304 - val_sparse_categorical_accuracy: 0.7065
Epoch 4/5
196/196 [==============================] - 7s 34ms/step - loss: 0.6519 - sparse_categorical_accuracy: 0.7710 - val_loss: 0.7688 - val_sparse_categorical_accuracy: 0.7395
Epoch 5/5
196/196 [==============================] - 7s 34ms/step - loss: 0.5415 - sparse_categorical_accuracy: 0.8103 - val_loss: 0.7833 - val_sparse_categorical_accuracy: 0.7474


In [9]:


model.save('mymodel.h5')



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:


with strategy.scope():
  model = tf.keras.models.load_model('mymodel.h5')
  out = model.predict(x_test[:1])
  print(out)



1/1 [==============================] - 2s 2s/step
[[-2.675701   -2.175026    1.1763275   6.4107103  -0.8578316   4.811289
   3.8336308  -2.2974846  -0.61419123 -2.0394797 ]]
